In [1]:
# import sys
# sys.path.append('/mnt/c/engDev/git_extrnl/pydantic')
%run __init__.py
import pathlib
import functools
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, date
from dataclasses import dataclass
from pydantic import BaseModel

import traitlets
import typing
from enum import Enum

import ipyautoui
from ipyautoui._utils import obj_from_string
from ipyautoui._custom_widgets import AutoUiDataGrid, AutoUiFileChooser
from ipyautoui.constants import DI_JSONSCHEMA_WIDGET_MAP, BUTTON_WIDTH_MIN

from enum import Enum
from ipyautoui.constants import BUTTON_WIDTH_MIN
from markdown import markdown
from datetime import datetime
import immutables

from enum import Enum
from ipyautoui.constants import BUTTON_WIDTH_MIN
from markdown import markdown
from datetime import datetime
import immutables
import json

In [2]:
   
#  -- ATTACH DEFINITIONS TO PROPERTIES ----------------------
def recursive_search_schema(sch:typing.Dict, li: typing.List) ->typing.Dict:
    """searches down schema tree to retrieve definitions

    Args:
        sch (typing.Dict): json schema made from pydantic   
        li (typing.List): list of keys to search down tree

    Returns:
        typing.Dict: definition retrieved from schema
    """
    if len(li) > 1:
        f = li[0]
        li_tmp = li[1:]
        sch_tmp = sch[f]
        return recursive_search_schema(sch_tmp, li_tmp)
    else:
        return sch[li[0]]

def update_property_from_definition(sch: typing.Dict, item: typing.Dict, key:typing.Any) -> typing.Dict:
    """attaches definition back to properties in schema

    Args:
        sch (typing.Dict): json schema
        item (typing.Dict): definition item
        key (typing.Any): what to search for (#ref)

    Returns:
        typing.Dict: sch
    """ 
    k = list(item.keys())[0]
    v = list(item.values())[0]
    
    li_filt = v[key].split('/')[1:]
    definition = recursive_search_schema(sch, li_filt)

    di_new = {}
    for k_, v_ in item.items():
        di_new[k_] = definition
    
    sch['properties'][k] = di_new[k]
    return sch

def update_property_definitions(sch: typing.Dict, key: str):
    """attaches all definitions back to properties. 
    TODO - currently only looks at the first level!

    Args:
        sch (typing.Dict): [description]
        key (str): [description]

    Returns:
        [type]: [description]
    """
    li_definitions = [{k:v} for k,v in sch['properties'].items() if key in v]
    for l in li_definitions:
         sch = update_property_from_definition(sch, l, key)
    return sch
#  ----------------------------------------------------------

#  -- CHANGE JSON-SCHEMA KEYS TO IPYWIDGET KEYS -------------
def update_key(key, di_map=DI_JSONSCHEMA_WIDGET_MAP):
    if key in di_map.keys():
        return di_map[key]
    else:
        return key
    
def update_keys(di, di_map=DI_JSONSCHEMA_WIDGET_MAP):
    return {update_key(k, di_map): v for k, v in di.items()}

def add_description_field(di):
    for k,v in di.items():
        if 'description' not in v:
            v['description'] =''
        # t=v['title']
        # d=v['description']
        # v['description'] = f"<b>{t}</b>, <i>{d}</i>"
        
    return di

def rename_schema_keys(di, di_map=DI_JSONSCHEMA_WIDGET_MAP):
    di = add_description_field(di)
    rename = {k:update_keys(v, di_map) for k, v in di.items()}
    return rename

def call_rename_schema_keys(di, di_map=DI_JSONSCHEMA_WIDGET_MAP, rename_keys=True):
    if rename_keys:
        return rename_schema_keys(di, di_map=di_map)
    else:
        return di
#  ----------------------------------------------------------

#  -- HELPER FUNCTIONS --------------------------------------
def get_type(pr, typ='string'):
    return {k:v for k,v in pr.items() if v['type'] ==typ}

def get_format(pr, typ='date'):
    pr = {k:v for k,v in pr.items() if 'format' in v}
    return {k:v for k,v in pr.items() if v['format'] ==typ}

def get_range(pr, typ='integer'):
    array = get_type(pr, typ='array')
    array = {k:v for k,v in array.items() if len(v['items']) ==2}
    tmp = {}
    for k,v in array.items():
        tmp[k] = v
        for i in v['items']:
            if 'minimum' not in i and 'maximum' not in i:
                tmp = {}
    if len(tmp)==0:
        return tmp
    else:
        rng = {k:v for k, v in tmp.items() if v['items'][0]['type'] == typ}
        for k, v in rng.items():
            rng[k]['minimum'] = v['items'][0]['minimum']
            rng[k]['maximum'] = v['items'][0]['maximum']
    return rng

def drop_enums(pr):
    return {k:v for k,v in pr.items() if 'enum' not in v}

def find_enums(pr):
    return {k:v for k,v in pr.items() if 'enum' in v}

def drop_explicit_autoui(pr):
    return {k:v for k,v in pr.items() if 'autoui' not in v}

def find_explicit_autoui(pr):
    return {k:v for k,v in pr.items() if 'autoui' in v}
#  ----------------------------------------------------------

#  -- FILTER FUNCTIONS --------------------------------------
#  -- find relevant inputs from json-schema properties ------
def get_IntText(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    ints = get_type(pr, typ='integer')
    simple_ints = {k:v for k,v in ints.items() if 'minimum' not in v and 'maximum' not in v}
    return call_rename_schema_keys(simple_ints, rename_keys=rename_keys)

def get_IntSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    ints = get_type(pr, typ='integer')
    simple_ints = {k:v for k,v in ints.items() if 'minimum' in v and 'maximum' in v}
    return call_rename_schema_keys(simple_ints, rename_keys=rename_keys)

def get_FloatText(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    floats = get_type(pr, typ='number')
    simple_floats = {k:v for k,v in floats.items() if 'minimum' not in v and 'maximum' not in v}
    return call_rename_schema_keys(simple_floats, rename_keys=rename_keys)

def get_FloatSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    floats = get_type(pr, typ='number')
    simple_floats = {k:v for k,v in floats.items() if 'minimum' in v and 'maximum' in v}
    return call_rename_schema_keys(simple_floats, rename_keys=rename_keys)

def get_Text(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    strings = get_type(pr)
    short_strings = drop_enums(strings)
    #short_strings = {k:v for k,v in strings.items() if 'maxLength' in v and v['maxLength']<200}
    return call_rename_schema_keys(short_strings, rename_keys=rename_keys)

def get_Textarea(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    strings = get_type(pr)
    simple_strings = drop_enums(strings)
    long_strings = {k:v for k,v in strings.items() if 'maxLength' in v and v['maxLength']>=200}
    return call_rename_schema_keys(long_strings, rename_keys=rename_keys)

def get_Dropdown(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    drops = find_enums(pr)
    drops = {k:v for k,v in drops.items() if v['type'] != 'array'}
    return call_rename_schema_keys(drops, rename_keys=rename_keys)

def get_SelectMultiple(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    mult = find_enums(pr)
    mult = {k:v for k,v in mult.items() if v['type'] == 'array'}
    return call_rename_schema_keys(mult, rename_keys=rename_keys)

def get_Checkbox(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_type(pr, typ='boolean'), rename_keys=rename_keys)

def get_DatePicker(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    date = get_type(pr, 'string')
    date = get_format(date)
    for k,v in date.items():
        if type(v['default']) == str:
            v['default'] = datetime.strptime(v['default'], "%Y-%m-%d").date()
    return call_rename_schema_keys(date, rename_keys=rename_keys)

def get_FileChooser(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    file = get_type(pr, 'string')
    file = get_format(file, typ='path')
    return call_rename_schema_keys(file, rename_keys=rename_keys)

def get_DataGrid(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    grid = get_type(pr, 'string')
    grid = get_format(grid, typ='DataFrame')
    return call_rename_schema_keys(grid, rename_keys=rename_keys)

def get_ColorPicker(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    color = get_type(pr, 'string')
    color = get_format(color, typ='color')
    return call_rename_schema_keys(color, rename_keys=rename_keys) 

def get_IntRangeSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_range(pr, typ='integer'), rename_keys=rename_keys)  

def get_FloatRangeSlider(pr, rename_keys=True):
    pr = drop_explicit_autoui(pr)
    return call_rename_schema_keys(get_range(pr, typ='number'), rename_keys=rename_keys)  

def get_AutoOveride(pr, rename_keys=True):
    pr = find_explicit_autoui(pr)
    return call_rename_schema_keys(pr, rename_keys=rename_keys)  
#  ----------------------------------------------------------

#  -- WIDGET MAPPING ----------------------------------------
#  -- uses filter functions to map schema objects to widgets 
def auto_overide(str_widget_type):
    return obj_from_string(str_widget_type)

class WidgetMapper(BaseModel):
    """defines a filter function and associated widget. the "fn_filt" is used to search the 
    json schema to find appropriate objects, the objects are then passed to the "widget" for the ui
    """
    fn_filt: typing.Callable
    widget: typing.Callable

DI_WIDGETS_MAPPER = {
    'IntText': WidgetMapper(fn_filt=get_IntText, widget=widgets.IntText),
    'IntSlider': WidgetMapper(fn_filt=get_IntSlider, widget=widgets.IntSlider),
    'FloatText': WidgetMapper(fn_filt=get_FloatText, widget=widgets.FloatText),
    'FloatSlider': WidgetMapper(fn_filt=get_FloatSlider, widget=widgets.FloatSlider),
    'Text': WidgetMapper(fn_filt=get_Text, widget=widgets.Text),
    'Textarea': WidgetMapper(fn_filt=get_Textarea, widget=widgets.Textarea),
    'Dropdown':WidgetMapper(fn_filt=get_Dropdown, widget=widgets.Dropdown),
    'SelectMultiple':WidgetMapper(fn_filt=get_SelectMultiple, widget=widgets.SelectMultiple),
    'Checkbox': WidgetMapper(fn_filt=get_Checkbox, widget=widgets.Checkbox),
    'DatePicker': WidgetMapper(fn_filt=get_DatePicker, widget=widgets.DatePicker),
    'AutoUiFileChooser': WidgetMapper(fn_filt=get_FileChooser, widget=AutoUiFileChooser),
    'AutoUiDataGrid': WidgetMapper(fn_filt=get_DataGrid, widget=AutoUiDataGrid),
    'ColorPicker': WidgetMapper(fn_filt=get_ColorPicker, widget=widgets.ColorPicker),
    'IntRangeSlider': WidgetMapper(fn_filt=get_IntRangeSlider, widget=widgets.IntRangeSlider),
    'FloatRangeSlider': WidgetMapper(fn_filt=get_FloatRangeSlider, widget=widgets.FloatRangeSlider),
    'AutoOveride': WidgetMapper(fn_filt=get_AutoOveride, widget=auto_overide),
}

def map_to_widget(sch: typing.Dict, di_widgets_mapper: typing.Dict=None) -> typing.Dict:
    """maps the widgets to the appropriate data using the di_widgets_mapper. 
    also renames json schema keys to names that ipywidgets can understand.

    Args:
        sch (typing.Dict): [description]
        di_widgets_mapper (typing.Dict, optional): [description]. Defaults to DI_WIDGETS_MAPPER.
            if new mappings given they extend DI_WIDGETS_MAPPER. it is expected that renaming 
            schema keys (call_rename_schema_keys) is done in the filter function

    Returns:
        typing.Dict: a dict (same order as original) with widget type
    """
    if di_widgets_mapper is None:
        di_widgets_mapper = DI_WIDGETS_MAPPER
    else:
        di_widgets_mapper = {**DI_WIDGETS_MAPPER, **di_widgets_mapper}
    sch = update_property_definitions(sch, '$ref')     
    pr = sch['properties']
    li_pr = pr.keys()
    di_ = {}
    for k, v in di_widgets_mapper.items():
        di = v.fn_filt(pr)
        for k_, v_ in di.items():
            di_[k_] = v_
            if 'autoui' not in v_:
                di_[k_]['autoui'] = v.widget
            else:
                di_[k_]['autoui'] = v.widget(v_['autoui'])          
    not_matched = set(di_.keys()) ^ set(li_pr) 
    if len(not_matched)>0:
        print('the following UI items from schema not matched to a widget:')
        print(not_matched)
    li_ordered = [l for l in li_pr if l not in not_matched]
    di_ordered = {l:di_[l] for l in li_ordered}        
    return di_ordered



In [3]:
if __name__ == "__main__":
    from ipyautoui.test_schema import TestAutoLogic
    from ipyautoui.constants import load_test_constants
    test_constants = load_test_constants()
    test = TestAutoLogic()
    sch = test.schema()
    mapped = map_to_widget(sch)
    [print(f"{k} --> {v['autoui']}") for k, v in mapped.items()]

the following UI items from schema not matched to a widget:
{'nested', 'array', 'select_multiple'}
string --> <class 'ipywidgets.widgets.widget_string.Text'>
int_slider --> <class 'ipywidgets.widgets.widget_int.IntSlider'>
int_text --> <class 'ipywidgets.widgets.widget_int.IntText'>
int_range_slider --> <class 'ipywidgets.widgets.widget_int.IntRangeSlider'>
float_slider --> <class 'ipywidgets.widgets.widget_float.FloatSlider'>
float_text --> <class 'ipywidgets.widgets.widget_float.FloatText'>
float_range_slider --> <class 'ipywidgets.widgets.widget_float.FloatRangeSlider'>
checkbox --> <class 'ipywidgets.widgets.widget_bool.Checkbox'>
dropdown --> <class 'ipywidgets.widgets.widget_selection.Dropdown'>
dropdown_simple --> <class 'ipywidgets.widgets.widget_selection.Dropdown'>
combobox --> <class 'ipywidgets.widgets.widget_string.Combobox'>
select_multiple_simple --> <class 'ipywidgets.widgets.widget_selection.SelectMultiple'>
text --> <class 'ipywidgets.widgets.widget_string.Text'>
text

In [5]:


def _init_widgets_and_rows(pr: typing.Dict)-> tuple((widgets.VBox, typing.Dict)):
    """initiates widget for from dict built from schema

    Args:
        pr (typing.Dict): schema properties - sanitised for ipywidgets

    Returns:
        (widgets.VBox, typing.Dict): box with widgets, di of widgets
    """
    di_widgets ={k:v['autoui'](**v) for k,v in pr.items()}
    labels = {k:widgets.HTML(f"<b>{v['title']}</b>, <i>{v['autoui_description']}</i>") for k,v in pr.items()}
    ui_box = widgets.VBox()
    rows = []
    for (k,v), (k2,v2) in zip(di_widgets.items(), labels.items()):
        rows.append(widgets.HBox([v,v2]))              
    ui_box.children = rows
    return ui_box, di_widgets
    
def file(self, path: pathlib.Path, **json_kwargs):
    """
    this is a method that is added to the pydantic BaseModel within AutoUi using 
    "setattr". i.e. setattr(self.config_autoui.pydantic_model, 'file', file)
    Args:
        self (pydantic.BaseModel): instance
        path (pathlib.Path): to write file to 
    """
    if 'indent' not in json_kwargs.keys():
        json_kwargs.update({'indent':4})
    path.write_text(self.json(**json_kwargs), encoding='utf-8')

class SaveControls(str, Enum):
    save_on_edit = 'save_on_edit'
    save_buttonbar = 'save_buttonbar'
    disable_edits = 'disable_edits'
    #archive_versions = 'archive_versions' #  TODO: implement this? 

class AutoUiConfig(BaseModel):
    pydantic_model: typing.Type[BaseModel]
    widgets_mapper: typing.Dict[str, WidgetMapper] = DI_WIDGETS_MAPPER
    save_controls: SaveControls = SaveControls.save_buttonbar
    ext: str ='.aui.json'
    
def save():
    print('save')
    
def revert():
    print('revert')

class SaveButtonBar():
    def __init__(self, save: typing.Callable=save, revert: typing.Callable=revert):
        self.fn_save = save
        self.fn_revert = revert
        self.out = widgets.Output()
        self._init_form()
        self._init_controls()
        
    def _init_form(self):
        self.unsaved_changes = widgets.ToggleButton(disabled=True, layout=widgets.Layout(width=BUTTON_WIDTH_MIN))
        self.revert = widgets.Button(icon='fa-undo',tooltip='revert to last save',button_style='warning',style={'font_weight':'bold'},layout=widgets.Layout(width=BUTTON_WIDTH_MIN))#,button_style='success'
        self.save = widgets.Button(icon='fa-save',tooltip='save changes',button_style='success',layout=widgets.Layout(width=BUTTON_WIDTH_MIN))
        self.message = widgets.HTML('a message')
        self.save_buttonbar = widgets.HBox([self.unsaved_changes, self.revert, self.save, self.message])
        
    def _init_controls(self):
        self.save.on_click(self._save)
        self.revert.on_click(self._revert)
        
    def _save(self, click):
        self.fn_save()
        self.message.value = markdown(f'_changes saved: {datetime.now().strftime("%H:%M:%S")}_')
        self._unsaved_changes(False)
        
    def _revert(self, click):
        self.fn_revert()
        self.message.value = markdown(f'_UI reverted to last save_')
        self._unsaved_changes(True)
        
    def _unsaved_changes(self, istrue: bool):
        if istrue:
            self.unsaved_changes.button_style='danger'
            self.unsaved_changes.icon='circle'
        else:
            self.unsaved_changes.button_style='success'
            self.unsaved_changes.icon='check'
        
    def display(self):
        with self.out:
            display(self.save_buttonbar)
        display(self.out)
        
    def _ipython_display_(self):
        self.display()
        
class AutoUi(traitlets.HasTraits):
    """AutoUi widget. generates UI form from pydantic schema. keeps the "value" field
    up-to-date on_change 

    Args:
        traitlets.HasTraits ([type]): traitlets.HasTraits makes it possible to observe the value
            of this widget.
    """
    value = traitlets.Dict()
    _pydantic_model: BaseModel =None

    def __init__(self, data: typing.Dict, config_autoui: AutoUiConfig=None, path: pathlib.Path=None):
        """init AutoUi

        Args:
            pydantic_obj (typing.Type[BaseModel]): initiated pydantic data object
            di_widgets_mapper (typing.Dict, optional): [description]. Defaults to DI_WIDGETS_MAPPER.
                if new mappings given they extend DI_WIDGETS_MAPPER. it is expected that renaming 
                schema keys (call_rename_schema_keys) is done in the filter function within di_widgets_mapper
        """
        self.config_autoui = config_autoui
        self.pydantic_obj = self.config_autoui.pydantic_model(**data)
        self.path = path
        self.save_on_edit = False
        self._init_ui()
        
    @property
    def pydantic_obj(self):
        return self._pydantic_obj
    
    @pydantic_obj.setter
    def pydantic_obj(self, pydantic_obj):
        self._pydantic_obj = pydantic_obj  # self.config_autoui.pydantic_model(**value) # set pydantic object
        self.value = self.pydantic_obj.dict()  # json.loads(self.pydantic_obj.json()) # set value 
        if hasattr( self, 'di_widgets' ):
            for k, v in self.value.items():
                if k in self.di_widgets.keys():
                    self.di_widgets[k].value=v
                else:
                    print(f'no widget created for {k}. fix this in the schema!')

        
    def _extend_pydantic_base_model(self):
        setattr(self.config_autoui.pydantic_model, 'file', file)

    def _init_ui(self):
        self._extend_pydantic_base_model()
        self.out = widgets.Output()
        self._init_schema()
        self._init_form()
        self._init_save_dialogue()
        self._init_controls()
        
    def _init_save_dialogue(self):
        assert self.path is not None, 'self.path == None. must be a valid path to save as json'
        map_save_dialogue = immutables.Map(save_on_edit=self.call_save_buttonbar,
                                           save_buttonbar=self.call_save_buttonbar,
                                           disable_edits=self.disable_edits)
        map_save_dialogue[self.config_autoui.save_controls]()
        
    def call_save_on_edit(self):
        self.save_on_edit = True
        
    def call_save_buttonbar(self):
        self.save_buttonbar = SaveButtonBar(save=self.file, revert=self._revert)
        self.ui_form.children = [self.save_buttonbar.save_buttonbar, self.title, self.ui_box]
           
    def disable_edits(self):
        for k,v in self.di_widgets.items():
            try: 
                v.disabled = True
            except:
                pass
                
    def _revert(self):
        assert self.path is not None, f'self.path = {self.path}. must not be None'
        self.pydantic_obj = self.config_autoui.pydantic_model.parse_file(self.path) # calls setter
        
    @classmethod
    def create_displayfile(cls, config_autoui: AutoUiConfig):
        return functools.partial(cls.parse_file, config_autoui=config_autoui)
        
    @classmethod
    def parse_file(cls, path: pathlib.Path, config_autoui: AutoUiConfig=None):
        if config_autoui is not None:
            cls.config_autoui = config_autoui
        assert cls.config_autoui is not None, ValueError('self.config_autoui must not be None')
        return AutoUi(cls.config_autoui.pydantic_model.parse_file(path).dict(), config_autoui=cls.config_autoui, path=path)

    def file(self, path: pathlib.Path=None):
        assert not self.path is None and path is None, f'self.path = {self.path}, path = {path}. no path given.'
        if self.path is not None and path is None:
            path = self.path
        ext = ''.join(path.suffixes)
        assert ext == self.config_autoui.ext, ValueError(f'the file extension should be: {self.config_autoui.ext}, but {ext} given. ')
        self.pydantic_obj.file(path)

    def _init_schema(self):
        sch = self.pydantic_obj.schema().copy()
        key = '$ref'
        self.sch = update_property_definitions(sch, key)
        self.pr = map_to_widget(self.sch, di_widgets_mapper=self.config_autoui.widgets_mapper)
    
    def _init_form(self):
        self.ui_form = widgets.VBox()
        self.title = widgets.HTML(f"<big><b>{self.sch['title']}</b></big> - {self.sch['description']}")
        self.ui_box, self.di_widgets = _init_widgets_and_rows(self.pr)
        self.ui_form.children = [self.title, self.ui_box]
        
    def _init_controls(self):
        [v.observe(functools.partial(self._watch_change, key=k), 'value') for k, v in self.di_widgets.items()];
        
    def _watch_change(self, change, key=None):
        setattr(self._pydantic_obj, key, self.di_widgets[key].value) # Note. not resetting the whole value so not using the setter
        self.value = self.pydantic_obj.dict() # json.loads(self.pydantic_obj.json()) # TODO: update this to .dict() once pydantic supports json-like serialisation to dicts not just strings
        if self.save_on_edit:
            self.file()
        if hasattr( self, 'save_buttonbar' ):
            self.save_buttonbar._unsaved_changes(True)
            
    def display(self):
        with self.out:
            display(self.ui_form)
        display(self.out)
        
    def _ipython_display_(self):
        self.display()
        
config_autoui = AutoUiConfig(ext='.aui.json', pydantic_model=TestAutoLogic, save_controls='save_buttonbar')
ui = AutoUi(test.dict(), config_autoui=config_autoui, path=test_constants.PATH_TEST_AUI)
ui

the following UI items from schema not matched to a widget:
{'nested', 'array', 'select_multiple'}


Output()

{'string': 'adsf',
 'int_slider': 2,
 'int_text': 1,
 'int_range_slider': [0, 2],
 'float_slider': 2.2,
 'float_text': 2.2,
 'float_range_slider': [0.0, 3.0],
 'checkbox': True,
 'dropdown': None,
 'dropdown_simple': 'asd',
 'combobox': 'asd',
 'select_multiple': ['male', 'female'],
 'select_multiple_simple': ['male', 'female'],
 'text': 'short text',
 'text_area': 'long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text ',
 'date_picker': '2021-11-30',
 'color_picker': '#c85151',
 'file_chooser': '.',
 'array': [],
 'datagrid': '{"test":{"0":0,"1":1

In [9]:
[display(v) for k,v in ui.di_widgets.items()]

Text(value='adsf')

IntSlider(value=2, max=3)

IntText(value=1)

IntRangeSlider(value=(0, 3), max=4)

FloatSlider(value=2.2, max=3.0)

...

DatePicker(value=datetime.date(2021, 11, 30))

ColorPicker(value='red')

AutoUiFileChooser(path='/mnt/c/engDev/git_mf/ipyautoui/src/ipyautoui', filename='', title='', show_hidden=Fals…

AutoUiDataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_…

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
ui.pydantic_obj.dict()

{'string': 'adsf',
 'int_slider': 2,
 'int_text': 1,
 'int_range_slider': (0, 3),
 'float_slider': 2.2,
 'float_text': 2.2,
 'float_range_slider': (0.0, 2.2),
 'checkbox': True,
 'dropdown': None,
 'dropdown_simple': 'asd',
 'combobox': 'asd',
 'select_multiple': [<Gender.male: 'male'>, <Gender.female: 'female'>],
 'select_multiple_simple': ['male', 'female'],
 'text': 'short text',
 'text_area': 'long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text ',
 'date_picker': datetime.date(2021, 11, 30),
 'color_picker': Color('red', rgb=(255, 0, 0)),
 'f

In [122]:
TestAutoLogic.parse_file(test_constants.PATH_TEST_AUI).dict()

{'string': 'adsfasdf',
 'int_slider': 2,
 'int_text': 1,
 'int_range_slider': (0, 3),
 'float_slider': 2.2,
 'float_text': 2.2,
 'float_range_slider': (0.0, 2.2),
 'checkbox': True,
 'dropdown': None,
 'dropdown_simple': 'asd',
 'combobox': 'asd',
 'select_multiple': [<Gender.male: 'male'>, <Gender.female: 'female'>],
 'select_multiple_simple': ['male', 'female'],
 'text': 'short text',
 'text_area': 'long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text long text ',
 'date_picker': datetime.date(2021, 11, 29),
 'color_picker': Color('red', rgb=(255, 0, 0)),

In [51]:
if __name__ == "__main__":
    from ipyautoui.constants import load_test_constants
    from ipyautoui.test_schema import TestAutoLogic
    from IPython.display import Markdown
    
    test_constants = load_test_constants()
    test = TestAutoLogic()
    config_autoui = AutoUiConfig(ext='.aui.json', pydantic_model=TestAutoLogic)
    ui = AutoUi(test.dict(), config_autoui=config_autoui)
    display(Markdown('# test display UI'))
    display(ui)
    display(Markdown('# test write to file'))
    ui.file(test_constants.PATH_TEST_AUI)
    print(f"test_constants.PATH_TEST_AUI.is_file() == {test_constants.PATH_TEST_AUI.is_file()} == {str(test_constants.PATH_TEST_AUI)}")
    display(Markdown('# test create displayfile widget'))
    TestAuiDisplayFile = AutoUi.create_displayfile(config_autoui=config_autoui)
    TestAuiDisplayFile(test_constants.PATH_TEST_AUI)

the following UI items from schema not matched to a widget:
{'array', 'select_multiple', 'nested'}


# test display UI

Output()

# test write to file

test_constants.PATH_TEST_AUI.is_file() == True == /mnt/c/engDev/git_mf/ipyautoui/tests/testdata/test.aui.json


# test create displayfile widget

the following UI items from schema not matched to a widget:
{'array', 'select_multiple', 'nested'}


In [89]:
from enum import Enum
from ipyautoui.constants import BUTTON_WIDTH_MIN
from markdown import markdown
from datetime import datetime

class SaveControls(str, Enum):
    save_on_edit = 'save_on_edit'
    save_buttonbar = 'save_buttonbar'
    disable_edits = 'disable_edits'
    #archive_versions = 'archive_versions' #  TODO: implement this? 

class AutoUiConfig(BaseModel):
    pydantic_model: typing.Type[BaseModel]
    widgets_mapper: typing.Dict[str, WidgetMapper] = DI_WIDGETS_MAPPER
    save_controls: SaveControls = SaveControls.save_buttonbar
    ext: str ='.aui.json'
    
def save():
    print('save')
    
def revert():
    print('revert')

class SaveButtonBar():
    def __init__(self, save: typing.Callable=save, revert: typing.Callable=revert):
        self.fn_save = save
        self.fn_revert = revert
        self.out = widgets.Output()
        self._init_form()
        self._init_controls()
        
    def _init_form(self):
        self.unsaved_changes = widgets.ToggleButton(disabled=True, layout=widgets.Layout(width=BUTTON_WIDTH_MIN))
        self.revert = widgets.Button(icon='fa-undo',tooltip='revert to last save',button_style='warning',style={'font_weight':'bold'},layout=widgets.Layout(width=BUTTON_WIDTH_MIN))#,button_style='success'
        self.save = widgets.Button(icon='fa-save',tooltip='save changes',button_style='success',layout=widgets.Layout(width=BUTTON_WIDTH_MIN))
        self.message = widgets.HTML('a message')
        self.save_buttonbar = widgets.HBox([self.unsaved_changes, self.revert, self.save, self.message])
        
    def _init_controls(self):
        self.save.on_click(self._save)
        self.revert.on_click(self._revert)
        
    def _save(self, click):
        self.fn_save()
        self.message.value = markdown(f'_changes saved: {datetime.now().strftime("%H:%M:%S")}_')
        self._unsaved_changes(False)
        
    def _revert(self, click):
        self.fn_revert()
        self.message.value = markdown(f'_UI reverted to last save_')
        self._unsaved_changes(True)
        
    def _unsaved_changes(self, istrue: bool):
        if istrue:
            self.unsaved_changes.button_style='danger'
            self.unsaved_changes.icon='circle'
        else:
            self.unsaved_changes.button_style='success'
            self.unsaved_changes.icon='check'
        
    def display(self):
        with self.out:
            display(self.save_buttonbar)
        display(self.out)
        
    def _ipython_display_(self):
        self.display()
        
save_dialogue = SaveButtonBar()
save_dialogue

In [76]:


now = datetime.now().strftime("%H:%M:%S")
now

'15:06:32'

In [75]:
self.show_me_the_code.button_style='info'

NameError: name 'self' is not defined

In [ ]:
@dataclass
class SimpleInputs:
    """NOT IN USE. defines simple input locations for SimpleEditJson"""
    fdir_inputs: str = 'fdir_inputs'
    fdir_template_inputs: str = 'fdir_template_inputs'
    fpth_inputs: str = 'fpth_inputs'
    fpth_inputs_options: InputOptions = InputOptions(
        project=InputDirs(fdir=fdir_inputs),
        template=InputDirs(fdir=fdir_template_inputs))

class FileController(Errors):

    def __init__(self, inputs: SimpleInputs):
        self.out = widgets.Output()
        self.inputs = inputs
        self._inherit_Inputs()
        self._init_FileController()

    def _init_FileController(self):
        self._errors()
        self.file_control_form()
        self._init_control()

    def _inherit_Inputs(self):
        self.__dict__.update(**asdict(self.inputs))

    def _init_control(self):
        self.save_changes.on_click(self._save_changes)
        self.load_inputs.observe(self._load_inputs,'value')
        self.load_button.on_click(self._load)
        self.revert.on_click(self._revert)

    @property
    def mf_layout(self):
        return widgets.Layout(
            display='flex',
            flex_flow='row',
            justify_content='flex-start',
            #border='dashed 0.2px green',
            grid_auto_columns='False',
            width='100%',
            #align_items='stretch',
        )

    def file_control_form(self):

        # button bar
        self.load_inputs = widgets.ToggleButton(icon='folder-open',tooltip='open inputs from file',button_style='info',layout=widgets.Layout(width='50px'))# (FontAwesome names without the `fa-` prefix)
        self.revert = widgets.Button(icon='fa-undo',tooltip='revert to last save',button_style='warning',style={'font_weight':'bold'},layout=widgets.Layout(width='50px'))#,button_style='success'
        self.save_changes = widgets.Button(icon='fa-save',tooltip='save changes',button_style='success',layout=widgets.Layout(width='50px'))
        self.button_bar = widgets.HBox([ self.load_inputs, self.revert, self.save_changes])

        # nested buttons
        template_inputs = self.get_template_inputs()
        project_inputs = self.get_project_inputs()
        options = dict(template_inputs, **project_inputs)
        self.load_button = widgets.Button(description = 'load',icon='fa-upload',style={'font_weight':'bold'})
        self.choose_inputs = widgets.RadioButtons(
            options = options,
            layout = self.mf_layout,
        )
        self.load = widgets.VBox([self.load_button,self.choose_inputs])
        self.temp_message = widgets.HTML(markdown('edit user input form below'))
        self.inputform = widgets.VBox([], layout = self.mf_layout)

    def _revert(self, sender):
        """revert to last save of working inputs file"""
        fpth = self.fpth_inputs
        self.temp_message.value = markdown('revert to inputs in last save of: {0}'.format(fpth))

        # add code here to revert to last save

        self.update_display()
        self.display()

    def _save_changes(self, sender):
        """save changes to working inputs file"""
        fpth = self.fpth_inputs
        dateTimeObj = datetime.now()
        self.save_timestampStr = dateTimeObj.strftime("%d-%b-%Y %H:%M:%S")
        self.temp_message.value = markdown('{0} saved at: {1}'.format(fpth, self.save_timestampStr))

        # add code here to save changes to file

        self.update_display()
        self.display()

    def _load_inputs(self,sender):
        """launches the inputs from file dialog"""
        self.temp_message.value = markdown('update the user input form with data from file')
        if self.load_inputs.value:
            self.inputform.children = [self.load_button, self.choose_inputs]
        else:
            self.temp_message.value = markdown('')
            self.inputform.children = []
        self.update_display()
        self.display()

    def _load(self,sender):
        self.update_display()
        self.display()
        fpth = self.choose_inputs.value

        # add code here to load form from file

        self.temp_message.value = markdown('input form load data from: {0}'.format(fpth))
        try:
            self.iserror = False
            message = 'load input form from: {0}'.format(fpth)
        except:
            self.iserror = True
            self.errormessage = '''__error__ loading : {0}.
            the file has either been deleted or modified such that it is unreadable'''.format(fpth)
        with self.out:
            clear_output()
            if self.load_inputs.value:
                display(Markdown(message))
            if self.iserror:
                display(Markdown(self.errormessage))

    def get_project_inputs(self):
        fpths = self.fpth_inputs_options['project']['fpths']
        di = {}
        for fpth in fpths:
            description = 'PROJECT: '+ fpth
            di[description] = fpth
        return di

    def get_template_inputs(self):
        fpths = self.fpth_inputs_options['template']['fpths']
        di = {}
        for fpth in fpths:
            description = 'TEMPLATE: '+ fpth
            di[description] = fpth
        return di

    def update_display(self):
        box = widgets.VBox([
            self.button_bar,
            self.temp_message,
            self.inputform,
        ])
        self.layout = box

    def display(self):
        display(self.layout)
        display(self.out)

    def _ipython_display_(self):
        self.update_display()
        self.display()